In [1]:
import nltk
import os
os.chdir('C://Users//HP//Desktop//datasets')
os.getcwd()


'C:\\Users\\HP\\Desktop\\datasets'

In [3]:

import numpy as np
import pandas as pd
d=pd.read_excel(io='C://Users//HP//Desktop//datasets//bppfinal2.xls')
d1=pd.read_excel(io='C://Users//HP//Desktop//datasets//schfinal2.xlsx')
d2=pd.read_excel(io='C://Users//HP//Desktop//datasets//ocdfinal2.xlsx')



In [4]:
subset = d[['Statement', 'Disorder']]
train = [tuple(x) for x in subset.values]

subset = d1[['Statement', 'Disorder']]
train1 = [tuple(x) for x in subset.values]

subset = d2[['Statement', 'Disorder']]
train2 = [tuple(x) for x in subset.values]


In [5]:
from nltk.stem import SnowballStemmer,WordNetLemmatizer
lem=WordNetLemmatizer()
ss=SnowballStemmer('english')


In [6]:
from nltk.tokenize import word_tokenize,sent_tokenize
words = set(word.lower() for passage in train for word in word_tokenize(passage[0]))
all_words=set(ss.stem(lem.lemmatize(w)) for w in words)
words1 = set(word.lower() for passage in train1 for word in word_tokenize(passage[0]))
all_words1=set(ss.stem(lem.lemmatize(w)) for w in words1)
words2 = set(word.lower() for passage in train2 for word in word_tokenize(passage[0]))
all_words2=set(ss.stem(lem.lemmatize(w)) for w in words2)

In [8]:
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in train]
t1 = [({word: (word in word_tokenize(x[0])) for word in all_words1}, x[1]) for x in train1]
t2 = [({word: (word in word_tokenize(x[0])) for word in all_words2}, x[1]) for x in train2]

In [9]:
import speech_recognition as sr
r=sr.Recognizer()
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source)
    print("Say something:")
    audio=r.listen(source)
test_sentence="" 
try:
    print("You said\n"+r.recognize_google(audio))  
except Exception as e:
    print(e)
test_sentence=test_sentence+r.recognize_google(audio)
print("Speech recognition completed")
sentences=sent_tokenize(test_sentence.lower())
sentences

Say something:
You said
today is a very good day and I am very proud of it and I am feeling extremely happy
Speech recognition completed


['today is a very good day and i am very proud of it and i am feeling extremely happy']

In [10]:
classifier = nltk.NaiveBayesClassifier.train(t)
classifier1 = nltk.NaiveBayesClassifier.train(t1)
classifier2 = nltk.NaiveBayesClassifier.train(t2)
classifier.show_most_informative_features()
classifier1.show_most_informative_features()
classifier2.show_most_informative_features()

Most Informative Features
                       , = True           bipola : normal =      7.1 : 1.0
                    feel = True           bipola : normal =      6.9 : 1.0
                      of = True           bipola : normal =      5.1 : 1.0
                      on = True           bipola : normal =      5.1 : 1.0
                    have = True           bipola : normal =      4.8 : 1.0
                     and = True           bipola : normal =      4.8 : 1.0
                     n't = True           bipola : normal =      4.4 : 1.0
                    that = True           bipola : normal =      4.2 : 1.0
                      am = True           normal : bipola =      4.2 : 1.0
                   about = True           bipola : normal =      3.7 : 1.0
Most Informative Features
                    that = True           Schizo : normal =     16.1 : 1.0
                       , = True           Schizo : normal =      7.8 : 1.0
                    what = True           Schizo

In [11]:
sentences=word_tokenize(str(sentences))
sentences


['[',
 "'today",
 'is',
 'a',
 'very',
 'good',
 'day',
 'and',
 'i',
 'am',
 'very',
 'proud',
 'of',
 'it',
 'and',
 'i',
 'am',
 'feeling',
 'extremely',
 'happy',
 "'",
 ']']

In [12]:
from nltk.corpus import stopwords
stop_words =set(stopwords.words('english'))
filtered_sentence=[w for w in sentences if not w in stop_words]

punctuations='''i!()-[]{};:'"\,<>./?@#$%^&*_~'''
filtered_sentence=[]
for w in sentences:
    if w not in stop_words and w not in punctuations:
        filtered_sentence.append(w)
        
print(sentences)
print(filtered_sentence)

['[', "'today", 'is', 'a', 'very', 'good', 'day', 'and', 'i', 'am', 'very', 'proud', 'of', 'it', 'and', 'i', 'am', 'feeling', 'extremely', 'happy', "'", ']']
["'today", 'good', 'day', 'proud', 'feeling', 'extremely', 'happy']


In [13]:
test_sent_features = {word.lower():(word in word_tokenize(ss.stem(lem.lemmatize(str(filtered_sentence))))) for word in filtered_sentence}
test_sent_features


{"'today": True,
 'day': False,
 'extremely': False,
 'feeling': False,
 'good': False,
 'happy': False,
 'proud': False}

In [14]:
print("bp:"+classifier.classify(test_sent_features))
print("sch:"+classifier1.classify(test_sent_features))
print("ocd:"+classifier2.classify(test_sent_features))

bp:normal
sch:normal
ocd:normal
